In [ ]:
!pip install langchain
!pip install openai
!pip install PyPDF2
!pip install faiss-cpu
!pip install tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 33.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.9/302.9 kB 23.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.0/121.0 kB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 14.8 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.0
    Uninstalling packaging-24.0:
      Successfully uninstalled packaging-24.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.6/320.6 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from PyPDF2 import PdfReader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS

In [ ]:
from typing_extensions import Concatenate
# Function to read text from PDF files
import PyPDF2

def read_pdf(file_paths):
    raw_text = ''
    for file_path in file_paths:
        with open(file_path, "rb") as f:
            reader = PyPDF2.PdfReader(f)
            raw_text+= str(file_path) + 'quaterly file report has started of TEL *****\n\n'
            for page in reader.pages:
                text = page.extract_text()
                raw_text += text
            raw_text+= str(file_path) + 'quaterly file report has started of TEL *****\n\n'
    return raw_text
#
# List of paths to PDF files
#pdf_files = ['Metro.pdf', 'Umar Farooq Khan.pdf']  # Add more file paths as needed

pdf_files = ["Q1FY23-Earnings-slides-TEL.pdf", 'Q2FY23-Earnings-slides-TEL.pdf']

# Read text from PDF files
raw_text = read_pdf(pdf_files)

#print(raw_text)


In [ ]:

# Split the text using Character Text Split such that it should not increase token size
text_splitter = CharacterTextSplitter(
    separator="\n",
    chunk_size=800,
    chunk_overlap=200,
    length_function=len,
)
texts = text_splitter.split_text(raw_text)

# Download embeddings from OpenAI
embeddings = OpenAIEmbeddings(openai_api_key='sk-proj-ggg')
document_search = FAISS.from_texts(texts, embeddings)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [ ]:
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import OpenAI

chain = load_qa_chain(OpenAI(openai_api_key='sk-proj-hhhhh'), chain_type="stuff")

query = "List out all the key points of guidance from Q1 report, what they claim to achieve for Q2?"
docs = document_search.similarity_search(query)
chain.run(input_documents=docs, question=query)

'\n1. Q1 FY23 Earnings Report has started for TEL.\n2. The presentation contains forward-looking statements and non-GAAP financial measures.\n3. Q1 sales and business performance were up, with reported sales increasing by 5% and organic sales increasing by 14%.\n4. Q1 adjusted operating margin was 18.2%, reflecting the timing of pricing actions to offset inflation.\n5. Organic net sales growth, adjusted operating margin, and adjusted EBITDA margin are all non-GAAP financial measures.\n6. The Industrial Solutions segment saw a 1% increase in sales and a 7% increase in organic growth.\n7. Margin expansion was driven by strong operational performance.\n8. The outlook for Q2 FY23 was provided on April 26, 2023.\n9. The report includes a description of non-GAAP financial measures.\n10. Medical delivered record quarterly sales, with increases in interventional procedures.\n11. Q2 sales and business performance were up, with reported sales increasing by 12% and organic sales increasing by 15%

In [ ]:
query = "Compare and contrast the guidance of Q1 with the actual outcomes of Q2 taken from Q2 report"
docs = document_search.similarity_search(query)
chain.run(input_documents=docs, question=query)

'\n\nIn Q1, the company had reported a 5% increase in sales and a 14% increase in organic sales. However, in Q2, the company reported quarterly sales of $4.16 billion, which was a 4% increase reported and an 8% increase organically year over year. This shows that while there was an increase in sales in both quarters, the growth rate decreased in Q2 compared to Q1.\n\nIn terms of business performance, in Q1, the company had reported a 5% increase in adjusted operating margin, while in Q2, the adjusted operating margin was reported to be 16.0%. This shows a significant improvement in Q2 compared to Q1.\n\nAdditionally, in Q1, the company had reported an adjusted EPS of $1.76, but in Q2, the adjusted EPS was $1.65. This shows a slight decrease in EPS from Q1 to Q2.\n\nIn terms of guidance for Q3, the company is expecting sales of approximately $4.0 billion and an adjusted EPS of $1.65, which is the same as the actual outcomes of Q2. This suggests that the company is expecting a similar pe

In [ ]:
query = "Which social impact goals does TEL claimed in Q1 but couldn achieve in Q2, and which goals and numbers did they exceed in a good sense?"
docs = document_search.similarity_search(query)
chain.run(input_documents=docs, question=query)

"\nTEL claimed in Q1 that they were named one of Fortune's World's Most Admired Companies for the 6th consecutive year, but they were not able to achieve this in Q2. However, they exceeded their Q2 sales and Adjusted EPS guidance, with sales of $4.16B and an 8% organic growth year-over-year."